In [23]:
import pandas as pd
import numpy as np

import plotly.express as px

from scipy.stats import shapiro

## Шаг 1. Загрузим данные.

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/taxi.csv', index_col=[0], parse_dates=[0])
data.head()

,num_orders
datetime,
2018-03-01 00:00:00,9
2018-03-01 00:10:00,14
2018-03-01 00:20:00,28
2018-03-01 00:30:00,20
2018-03-01 00:40:00,32


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 26496 entries, 2018-03-01 00:00:00 to 2018-08-31 23:50:00
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   num_orders  26496 non-null  int64
dtypes: int64(1)
memory usage: 414.0 KB


Проверим данные на монотонность

In [4]:
if data.index.is_monotonic_increasing:
    print('Монотонный временной ряд, данные распределены от прошлого к будущему.')
else:
    print('Данные в датасете перемешаны во времени.')

Монотонный временной ряд, данные распределены от прошлого к будущему.


Проверим, что в данных нет одинаковых моментов времени

In [11]:
if data.index.is_unique:
    print('Временные метки уникальны')
else:
    print('Временные метки не уникальные')

Временные метки уникальны


In [15]:
data_by_hour = data.resample('1h').sum()

In [19]:
fig = px.histogram(
    data_by_hour,
    title='Распределение количества заказов'
)

fig.update_xaxes(
    title='Количество заказов '
)

fig.update_yaxes(
    title='Количество'
)

fig.update_layout(
    showlegend=False,
    bargap=0.1,
)

fig.show()

In [26]:
d = shapiro(data_by_hour)

if d[1] < 0.05:
    print('MMore')

MMore
